In [1]:
import os, sys
from pathlib import Path

ROOT = Path.cwd()
if not (ROOT / "app").exists() and (ROOT.parent / "app").exists():
    ROOT = ROOT.parent  # handle notebooks/ cwd
sys.path.insert(0, str(ROOT))
os.environ["PYTHONPATH"] = str(ROOT)


# Feature Playground (Targeted Checks)

Use this notebook to sanity-check individual pieces of the MVP while the FastAPI server runs via `uvicorn server:app`. Full end-to-end demo steps live in `DEMO_GUIDE.md`.

In [2]:
import os
import json
from pathlib import Path

import httpx
from dotenv import load_dotenv

load_dotenv(Path.cwd() / "app" / ".env")
BASE_URL = os.getenv("API_BASE_URL", "http://localhost:3000/api")
HEADERS = {}
if os.getenv("INTERNAL_API_KEY"):
    HEADERS["X-Internal-Key"] = os.environ["INTERNAL_API_KEY"]
client = httpx.Client(base_url=BASE_URL, headers=HEADERS)
print(f"🌐 Base URL: {BASE_URL}")
print(f"🔐 Sending internal key: {'X-Internal-Key' in HEADERS}")

🌐 Base URL: http://localhost:3000/api
🔐 Sending internal key: True


## 1. Yahoo Finance data check
Fetch live prices/indicators for a few tickers using `yfinance` so you can validate the raw inputs shown to agents.

In [3]:
import yfinance as yf

symbols = ["AAPL", "MSFT", "NVDA"]
rows = []
for symbol in symbols:
    ticker = yf.Ticker(symbol)
    info = ticker.history(period="5d").tail(1)
    if info.empty:
        rows.append({"symbol": symbol, "price": None, "volume": None})
        continue
    latest = info.iloc[0]
    rows.append({
        "symbol": symbol,
        "price": round(float(latest["Close"]), 2),
        "volume": int(latest["Volume"]),
    })

print(json.dumps(rows, indent=2))

[
  {
    "symbol": "AAPL",
    "price": 273.81,
    "volume": 17910600
  },
  {
    "symbol": "MSFT",
    "price": 488.02,
    "volume": 5855900
  },
  {
    "symbol": "NVDA",
    "price": 188.61,
    "volume": 65528500
  }
]


## 2. LLM decision check
Call the orchestrator directly (prefers DeepSeek if configured, otherwise OpenAI) to confirm API keys work and responses can be parsed.

In [5]:
import asyncio
from app.core.llm_service import LLMOrchestrator
from app.core.trading_styles import TRADING_STYLES

market_snapshot = """
AAPL $210 (RSI 65)
MSFT $400 (RSI 55)
NVDA $125 (RSI 72)
""".strip()
account_state = {"cash": 10000, "positions": {}}
orchestrator = LLMOrchestrator()

async def run_llm_probe():
    preferred_model = "gpt-4o-mini"  # whatever you want to test
    style = "momentum"
    decision = await orchestrator.get_decision(
        model_name=preferred_model,
        system_prompt=TRADING_STYLES[style]["system_prompt"],
        market_data=market_snapshot,
        account_state=json.dumps(account_state),
    )
    return {
        "model": preferred_model,
        "style": style,
        "decision": decision.action,
        "ticker": decision.ticker,
        "quantity": decision.quantity,
        "confidence": decision.confidence,
    }

probe_result = await run_llm_probe()  # <— replace asyncio.run(...)
print(json.dumps(probe_result, indent=2))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
  "model": "gpt-4o-mini",
  "style": "momentum",
  "decision": "buy",
  "ticker": "AAPL",
  "quantity": 20,
  "confidence": 0.8
}


## 3. API storage check
With the server running, hit the REST API to register a user, select agents, store a trade, and read it back.

In [6]:
from datetime import datetime

# Register throwaway user
user_payload = {
    "username": f"playground-{datetime.utcnow().timestamp():.0f}",
    "email": f"demo+{datetime.utcnow().timestamp():.0f}@example.com",
}
user = client.post("/users/register", json=user_payload).json()
print("👤 User", json.dumps(user, indent=2))

# Grab first three agents
agents = client.get("/agents").json()
selection = [agent["id"] for agent in agents[:3]]
client.post(f"/users/{user['user_id']}/select-agents", json={"agent_ids": selection})
print("🤝 Selected agents", selection)

# Record a sample trade under the first agent
trade_payload = {
    "week_number": 1,
    "action": "buy",
    "stock_ticker": "PLAY",
    "quantity": 2,
    "price": 123.45,
    "pnl_delta": 0.0,
    "action_reason": "Playground sanity trade",
    "user_id": user["user_id"],
}
client.post(f"/agents/{selection[0]}/trades", json=trade_payload)

# Fetch stored trades + user P&L
trades = client.get(f"/agents/{selection[0]}/trades").json()
pnl = client.get(f"/users/{user['user_id']}/pnl").json()
print("📄 Trades", json.dumps(trades, indent=2))
print("💰 User P&L", json.dumps(pnl, indent=2))

/var/folders/m_/cm0g2c1d31x1h9s7s2cw20s00000gn/T/ipykernel_12005/2995134667.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "username": f"playground-{datetime.utcnow().timestamp():.0f}",
/var/folders/m_/cm0g2c1d31x1h9s7s2cw20s00000gn/T/ipykernel_12005/2995134667.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "email": f"demo+{datetime.utcnow().timestamp():.0f}@example.com",
INFO:httpx:HTTP Request: POST http://localhost:3000/api/users/register "HTTP/1.1 201 Created"
INFO:httpx:HTTP Request: GET http://localhost:3000/api/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000/api/users/1/select-agents "HTTP/1.1 201 Created"
INFO:httpx:HTTP Reques

👤 User {
  "user_id": 1,
  "username": "playground-1766757107",
  "email": "demo+1766757107@example.com",
  "created_at": "2025-12-26T05:51:46.735653"
}
🤝 Selected agents [1, 2, 3]
📄 Trades {
  "agent_id": 1,
  "trades": [
    {
      "event_id": 1,
      "timestamp": "2025-12-26T05:51:46.747714",
      "pnl_delta": 0.0,
      "agent_id": 1,
      "user_id": 1,
      "week_number": 1,
      "action": "buy",
      "stock_ticker": "PLAY",
      "quantity": 2,
      "price": 123.45,
      "action_reason": "Playground sanity trade"
    }
  ]
}
💰 User P&L {
  "user_id": 1,
  "agents": [
    {
      "agent_id": 1,
      "total_pnl": 0.0,
      "trade_count": 1
    },
    {
      "agent_id": 2,
      "total_pnl": 0.0,
      "trade_count": 0
    },
    {
      "agent_id": 3,
      "total_pnl": 0.0,
      "trade_count": 0
    }
  ],
  "total_pnl": 0.0
}


In [20]:

import os, json, datetime, asyncio, yfinance as yf
from datetime import timezone
from app.core.llm_service import LLMOrchestrator
from app.core.trading_styles import TRADING_STYLES

# Helper to make POSTs and fail loudly if response is not JSON

def post_json(path, payload, timeout=90):
    resp = client.post(path, json=payload, timeout=timeout)
    if resp.status_code >= 400:
        print(f"HTTP {resp.status_code} for {path}" + resp.text[:1000])
        raise RuntimeError(f"Request failed: {resp.status_code}")
    try:
        return resp.json()
    except Exception:
        print("Non-JSON response:", resp.text[:1000])
        raise

orchestrator = LLMOrchestrator()

# Live prices to nudge momentum agents into action
symbols = ["AAPL", "NVDA", "MSFT"]
prices = {
    sym: float(yf.Ticker(sym).history(period="1d")["Close"].iloc[-1])
    for sym in symbols
}
market_snapshot = f"""
AAPL ${prices['AAPL']:.2f} (RSI 72, MACD positive, strong uptrend)
NVDA ${prices['NVDA']:.2f} (RSI 75, breakout momentum, MACD rising)
MSFT ${prices['MSFT']:.2f} (RSI 68, steady uptrend, MACD positive)
Cash available: $10,000
""".strip()
account_state = {"cash": 10000, "positions": {sym: 0 for sym in symbols}}

have_openai = bool(os.getenv("OPENAI_API_KEY"))
have_deepseek = bool(os.getenv("DEEPSEEK_API_KEY"))
print(f"OpenAI key present: {have_openai}, DeepSeek key present: {have_deepseek}")

# Pick momentum agents (OpenAI + DeepSeek) and pad roster to 3+ for selection
catalog = client.get("/agents").json()
momentum_agents = [a for a in catalog if a.get("style_name") == "momentum"]
openai_momentum = next((a for a in momentum_agents if a.get("model_name") == "gpt-4o-mini"), None) if have_openai else None
deepseek_momentum = next((a for a in momentum_agents if a.get("model_name") == "deepseek-chat"), None) if have_deepseek else None

fallback_agent = next((a for a in catalog if a not in (openai_momentum, deepseek_momentum)), None)
selection_ids = []
for a in (openai_momentum, deepseek_momentum, fallback_agent):
    if a and a.get("id") not in selection_ids:
        selection_ids.append(a["id"])
for a in catalog:
    if len(selection_ids) >= 3:
        break
    if a.get("id") not in selection_ids:
        selection_ids.append(a["id"])

trading_agents = [a for a in (openai_momentum, deepseek_momentum) if a]
if not trading_agents:
    raise RuntimeError("No LLM agents available; set OPENAI_API_KEY and/or DEEPSEEK_API_KEY")

user = post_json(
    "/users/register",
    {"username": f"live-llm-{int(datetime.datetime.now(timezone.utc).timestamp())}", "email": "live-llm@example.com"},
    timeout=30,
)
user_id = user["user_id"]
post_json(f"/users/{user_id}/select-agents", {"agent_ids": selection_ids}, timeout=30)


async def fetch_decision(agent):
    try:
        style_name = agent["style_name"]
        system_prompt = TRADING_STYLES[style_name]["system_prompt"]
        decision = await orchestrator.get_decision(
            model_name=agent["model_name"],
            system_prompt=system_prompt,
            market_data=market_snapshot,
            account_state=json.dumps(account_state),
        )
        payload = {
            "action": decision.action,
            "ticker": decision.ticker,
            "quantity": decision.quantity,
            "confidence": decision.confidence,
            "reasoning": decision.reasoning,
        }
        print(f"Agent {agent['id']} ({agent['model_name']} / {agent['style_name']}):")
        print(json.dumps(payload, indent=2))
        return payload
    except Exception as e:
        print(f"LLM error for agent {agent['id']}: {e}")
        return None


def record_trade(agent, decision):
    if not decision:
        return
    action = decision.get("action", "hold")
    if action == "hold":
        print("Model said hold; not recording a trade.")
        return
    ticker = (decision.get("ticker") or "AAPL").upper()
    price = prices.get(ticker, list(prices.values())[0])
    trade = {
        "week_number": datetime.date.today().isocalendar().week,
        "action": action,
        "stock_ticker": ticker,
        "quantity": decision.get("quantity") or 10,
        "price": price,
        "pnl_delta": 0.0,
        "action_reason": decision.get("reasoning", "LLM decision"),
        "user_id": user_id,
    }
    saved = post_json(f"/agents/{agent['id']}/trades", trade, timeout=30)
    print("Recorded trade:", json.dumps(saved, indent=2))


async def main():
    for agent in trading_agents:
        dec = await fetch_decision(agent)
        record_trade(agent, dec)

await main()


INFO:httpx:HTTP Request: GET http://localhost:3000/api/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000/api/users/register "HTTP/1.1 201 Created"
INFO:httpx:HTTP Request: POST http://localhost:3000/api/users/12/select-agents "HTTP/1.1 201 Created"


OpenAI key present: True, DeepSeek key present: True


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


LLM error for agent 3: 'list' object has no attribute 'get'


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000/api/agents/8/trades "HTTP/1.1 201 Created"


Agent 8 (deepseek-chat / momentum):
{
  "action": "buy",
  "ticker": "NVDA",
  "quantity": 53,
  "confidence": 0.85,
  "reasoning": "NVDA shows strongest momentum signals: RSI 75 indicates strong bullish momentum, MACD rising confirms trend acceleration, and breakout momentum suggests continuation. Position size of ~$10,000 (25% of portfolio) aligns with momentum strategy. Entry criteria met with RSI > 60 and strong uptrend. Will implement 10% stop loss at $169.75 to limit risk to 4% of position."
}
Recorded trade: {
  "event_id": 4,
  "timestamp": "2025-12-26T06:45:28.405674",
  "pnl_delta": 0.0,
  "agent_id": 8,
  "user_id": 12,
  "week_number": 52,
  "action": "buy",
  "stock_ticker": "NVDA",
  "quantity": 53,
  "price": 188.61000061035156,
  "action_reason": "NVDA shows strongest momentum signals: RSI 75 indicates strong bullish momentum, MACD rising confirms trend acceleration, and breakout momentum suggests continuation. Position size of ~$10,000 (25% of portfolio) aligns with mo